# 実際に回帰問題を解いてみる

## 回帰問題編

解説付きでコードを示しています．使い方としては

1. 上から順に読んで（重要），
2. セルを実行していってください．
3. 理解できない場合は，スタッフに質問を投げるなどしてください．
4. 理解が進んだら，自分のノートを作成し，ページにコードを真似して記述していってください．

In [ ]:
# まずは必要そうなものをインポート scipy は要らないかも
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns; sns.set()

ここでは直線のモデル $y = a x + b$ を考えます．

データは，直線モデルから，ノイズが加えられて観測したものと仮定し，人工的に生成します．
要は，機械学習が出すべき答えを知っている状態で，データを適用しどのような解が得られるのかを実験することが目的となります．

詳細はコードに書いていますが，データを作成する場合は，直線モデルを具体的に与え(a = 1.2, b=0.5 などとして)，
ノイズの重畳過程も記載することが必要です．
ここではノイズの重畳は加法的ガウスノイズが重畳されるとし，ノイズの標準偏差は sgm = 0.2 としています．

In [ ]:
# ここらへんのパラメータを変えて遊んでみてください

N = 50  #サンプル点の個数
a = 1.2  #モデル直線の傾き
b = 0.5  #モデル直線の切片
sgm = 0.2  # ノイズの標準偏差

x = np.random.rand(N)    # [0, 1) の区間に N 個の乱数発生
ytrue = a * x + b

y = ytrue  + sgm * np.random.randn(N)  # 真値にノイズを乗せて観測データを作る

### これで下準備完了

一応，解説を入れておくと

* `x`: データのx座標（乱数で生成） 
* `ytrue`: a x + b 上の値 
* `y`: `ytrue` にガウスノイズを載せたもの 

です．
なので，(x, y) にデータが入っていると思いましょうという話です．
そこで，これらの点のプロットと真の直線の関係を見てみることにします．

In [ ]:
# 散布図の描画
plt.figure()
plt.plot(x, y, 'bo', label='data')
plt.xlim(0, 1)
plt.ylim(0.4, 1.8)

このような青点の散布図が与えられたときに

In [ ]:
# 真の関係を描画
plt.figure()
plt.plot(x, y, 'bo', label='data')
plt.xlim(0, 1)
plt.ylim(0.4, 1.8)
xx = np.linspace(0, 1, 128)
yy = a * xx + b
plt.plot(xx, yy, 'r-', label='true')
plt.legend()

上図のような赤線を得ることができるのか？というのが問いかけになります．
少しかっこいい言葉遣いをすれば

### 問い: データ点（青い点）のみから，もとになる直線（赤直線）を推定することは可能か？

になります．これには解法としていくつか考えられますが，ここでは以下のような機械学習と呼ばれる手法に頼ることとします．

データの点群を $\{(x_n, y_n)\}$ とし， 推定モデルを $f(x; w) = w_1 x + w_0$ とおいて， $w_0$ と $w_1$ を推定することを考えます．
この問題は *最小二乗法* の問題で，　以下の関数（ロス関数と呼ばれる）
$$
    J(w) = \frac{1}{N} \sum_n (y_n - f(x_n))^2
$$
を，最小化する $w$ を求めます．

### 要はモデル $f(x_n)$ と，　観測点 $y_n$ の差（残差）が小さくなる $w$ を求める．

ことがやりたいことになります．

この問題は，正規方程式
$$
    \left(\begin{array}{rr} N & \sum x_n \\ \sum x_n  & \sum x_n^2 \end{array}\right) 
    \left(\begin{array}{r} w_0 \\ w_1 \end{array}\right)
    =
     \left(\begin{array}{r} \sum y_n \\ \sum x_n y_n \end{array}\right)
$$
を解けばよい（導出が知りたければ質問すること）ことになります．
解は下記のとおりです．
$$
    \left(\begin{array}{r} w_0 \\ w_1 \end{array}\right)
    =
    \frac{1}{N \sum x_n^2 - (\sum x_n)^2}
    \left(\begin{array}{rr} \sum x_n^2  & - \sum x_n \\ - \sum x_n  & N\end{array}\right)
     \left(\begin{array}{r} \sum y_n \\ \sum x_n y_n \end{array}\right)
$$

In [ ]:
# 最小二乗法から解をもとめよ

#とりあえず統計量を計算しておく（上式の和記号が付いたやつ）
xsum = x.sum()
x2sum = (x**2).sum()
ysum = y.sum()
xysum = x @ y

# 2x2 の逆行列は手計算で解ける
det = N * x2sum - (xsum)**2
w0 = (x2sum * ysum - xsum * xysum) / det
w1 = (-xsum * ysum + N * xysum) / det

print("Estimate w1, w0 = ({:.3f}, {:.3f})".format(w1, w0))
print("True      a,  b = ({:.3f}, {:.3f})".format(a, b))

In [ ]:
# 得られた直線が正しそうかプロット してみよう

plt.plot(x, y, 'bo', label='data')
plt.xlim(0, 1)
plt.ylim(0.4, 1.8)

xx = np.linspace(0, 1, 128)
yy = a * xx + b
plt.plot(xx, yy, 'r-', label='true')
yy = w1 * xx + w0
plt.plot(xx, yy, 'g-', label='estimate')
plt.legend() # 凡例を描画

緑のラインが推定で，赤のラインが真値．

ついでに，残差とロス関数値も評価しておこう．　残差は モデル $f(x) = w_1 x + w_0$ が吐き出す予測値と $y$ の差の総和

In [ ]:
# 残差を表示せよ

residual = np.sum((y - (w1 * x + w0))**2)
print( "Residual: {:.3f}".format(residual))
print( "Mean squared error {:.3f}".format(residual/N))   #1点あたりのズレ（平均二乗誤差）

全体の残渣がおおよそ 2.5 なので，1点あたりのズレは 0.052 となり

### うんだいたい良さげ

ということがわかります．

でも，正規方程式とか面倒です．．．で，この演習では python のパッケージを使った機械学習手法を習得しましょうという話になります．
紹介コードとしては，以下の２種を提供します．

1. scikit-learn を使う
2. PyTorch を使う

このレベルの問題であれば１を使うのが楽ですが，後半の画像認識では，選択肢２がメインになるので，一応両方とも解法を記載しておきます．
なお，モデルは一緒でも 1 は，上述の行列を用いて解きますが，2 は勾配法を用いて解きますので，解へのアプローチが異なります．

In [ ]:
# sklearn を使った解答

from sklearn import linear_model   # 線形モデル導入
from sklearn.metrics import mean_squared_error  # 平均二乗誤差

# モデルは，回帰(regression) なので regr というオブジェクトとして扱う
regr = linear_model.LinearRegression()

#線形回帰モデル(Linear Regression()) が期待しているデータ形式は，各行にデータが入っているものなので reshape して渡す
regr.fit(x.reshape(N, 1), y.reshape(N, 1))

フィットさせることに成功した場合，なにもえらーは吐き出しません．
モデルを定義して, モデル内の fit() 関数を呼び出すだけでOKですが，知りたいのはモデルのパラメータなどです．
fit したあとは， モデル内の coef_ と intercept_ を見ればそこに解が入っています．

In [ ]:
# regression の coef_ と intercept_ は
# y = A x + b の A と b に対応．
# 成分でかくと y_i = \sum_i A_ij x_j + b_i な感じ
# なので成分を出すためには coef_[i][j] とb[i] を指定する必要がある．

print("w0, w1 = ({:.3f}, {:.3f})".format(regr.coef_[0][0], regr.intercept_[0]))

## PyTorch を用いた解法

PyTorch は，Facebook （現Meta）のもとで開発されてきた深層学習要の枠組み（フレームワーク）である．
（現在は Linux Foundation の下にいるっぽい）
深層学習の枠組みは様々なものがあったけど，Google の TensorFlow が死にそうなのでこちらを使って解いてみる．

深層学習用のパッケージなので，普通は，
## こんな単純な回帰問題には *使わない* ．
けど理解するにはちょうど良いかもしれない．

後半との連結をよくするために雰囲気で書いています．
（まじめに線形回帰するだけならscikit-learnのほうがうまく動きます）

PyTorch でも

1. 必要なオブジェクトのインポート
2. モデルの構築
3. fit（最適化）

という手順の流れは一緒です．

まず，モデルとはなにかというところから考えます．

上述の説明では，回帰モデルは 
$$f(x; w) = w_1 x + w_0$$
です．パラメータは $w_1$ と $w_0$ な形で表されます．
モデルへの入力は $x$ で，出力が $f(x; w)$ となります．

さて，これをニューラルネットワークで表現することを考えます．
ニューラルネットは，誤解を恐れずに言えば，丸（節点）と線（辺）でかく計算モデル（グラフ）です．
下図左の数式モデルを計算グラフで表すと右図のようになります．

<img src="Images/modelLinear.png" width=50%>

入力 $x$ を持っている節点は出力ノード（オレンジ色のノードに）値$x$ を渡します．そのときに辺にかかっている
パラメータ($w_1$)を掛けて渡します．また１が入っている節点は１を出力するノードで，同じように辺の値($w_0$) を
かけて渡します．オレンジ色のノードは，渡された重み付きの入力を加算し，場合によっては非線形変換を行って，出力を出します．
この操作を各節点で順次行っていき最終的な出力を得ます．
このように入力を快層状に変換していく方法なので，階層型ニューラルネットワークと呼ばれます．

ということで，この計算モデルをまず作ってみましょう．
PyTorch では，計算モデルはオブジェクトとして作成するのが一般的で，これは ``torch.nn.Module`` を継承することでのが一般的です．
クラス定義としては，最小限以下の２つのメソッドを用意します．

* ``__init__()``: コンストラクタ．オブジェクトを生成する際に呼ばれる．ここで階層構造を作ります．
* ``forward()``: 順方向（入力から出力へ）の計算過程

モデルとしては，入力 $x$ １個と，出力 $f(x; w)$ 1 個です．常に１という出力を出すユニットは 'バイアス項' と呼ばれ，入力変数としては扱わない場合が一般的です．
線形ユニットの記述は ``nn.Linear(出力個数, 入力個数, バイアス項の有無)`` で，
この設計を ``__init__()`` を埋め込みます．この場合 ``nn.Linear(1, 1, bias=True)`` となります．

``forward()`` 計算では，階層構造に入力を入れて出てきた値を返す形でOKです．

In [ ]:
import torch
from torch import nn

class LinearRegression(nn.Module):
    '''
    １入力１出力を行う線形回帰モデル
    '''
    def __init__(self):
        '''コンストラクタ'''
        super().__init__() # 継承元のコンストラクタを呼び出し
        # ネットワーク構造の記述，１入力１出力，バイアス項あり
        self.layer = nn.Linear(1, 1, bias=True) 
    
    def forward(self, x):
        '''順方向の計算，入力xを出力y へ'''
        # 線形変換のみで記述
        y = self.layer(x)
        return y


モデルが決まったら，後は
* モデルのインスタンス化
* 損失関数を決めて
* 最適化を学習データに対して順次行う
ということをやります．モデルのインスタンス化は上記で定義した ``LinearRegression`` クラスを実体化することで行います．
損失関数は二乗誤差で定義しているので，``nn.MSEloss()`` で規定します．
最適化に関してはスタンダードな，確率勾配法を用います．

勾配法がやっていることは，与えられた入力に対して微分をとって，その値が小さくなるように購買を下るように重みパラメータ $w_0, w_1$ を動かすことになります．

In [ ]:
# モデルをインスタンス化
model = LinearRegression()

# 損失関数の定義
loss_criterion = nn.MSELoss()

# 最適化手法
learning_rate = 0.01 # 学習率（1回にどのくらい更新するか）
moment_rate = 0.9    # 慣性項（1epoch 前の更新をどのくらい引きずるか）
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=moment_rate)

頑張って，データに対してループを回していきます

In [ ]:
# epoch は何セット学習データを見せるか N 回データを通すと 1 epoch

num_epoch = 1000

# 入力は 32bit 浮動小数点を要求されるので ``torch.float32`` へ変換しておく
inputs = torch.from_numpy(x.reshape(-1, 1)).to(torch.float32)
targets = torch.from_numpy(y.reshape(-1, 1)).to(torch.float32)

# ロスの記録用
hist = []

for epoch in range(num_epoch):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = loss_criterion(outputs, targets) # ずれの量をはかる

    # ズレからパラメータを更新
    loss.backward()
    optimizer.step()

    hist.append(loss.item())
    # 更新してどのくらい良くなったかを表示（100回に１回程度の頻度で）
    if (epoch+1) % 100 == 0:
        print(f'Epoch[{epoch+1:02d}/{num_epoch:02d}]: loss = {loss.item():04f}')


# 結果の状態を保存したい場合は下をコメントアウトする．
# torch.save(model.state_dict(), 'model1.pickl')

うまくロス関数が小さくなっているかを確認する．

In [ ]:
# ロスの値が学習によってどう変わっていくのかを表示してみる

plt.plot(np.array(hist))
#plt.semilogy(np.array(hist)) #対数スケールの方が収束したかは判断しやすい
plt.title('Loss Evolution')
plt.xlabel('Epochs')
plt.ylabel('loss')


かなり頑張って前出のロス値付近に収束しているぽい．
なので，パラメータを取り出して確認してみる

In [ ]:
w1, w0 = model.layer.weight[0][0], model.layer.bias[0]
print( "w0, w1 = ({:.3f}, {:.3f})".format(w0, w1))

In [ ]:
#w0 と w1 は， np.array なので，使いやすいように float 型へ変換しておく
w0 = float(w0)
w1 = float(w1)

# あとはプロット
plt.plot(x, y, 'bo')
plt.xlim(0, 1)
plt.ylim(0.4, 1.8)

xx = np.linspace(0, 1, 128)
yy = a * xx + b
plt.plot(xx, yy, 'r-')
yy = w1 * xx + w0
plt.plot(xx, yy, 'g-')

多分，前述の解とは微妙にずれているが，そんなに致命的ではないのを確認してください.
でも，`epoch` をもうちょっと増やして，頑張ると落とせるとは思います．
#### 逆説的に言えば，機械学習による推定とは，そのくらいの誤差が含まれるものと認識してください．